In [22]:
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

True

In [23]:
SYSTEM_PROMPT = """
You are an Australia high schoold teacher.
"""
# LABELER_PROMPT = """You are an Australia high schoold teacher. You are required to do the jobs to assist with student teaching, follow the instructions carefully.
#     - You are given a math question and solution, you need to fill in the information for the question and output in Json format:
#     - You should give the level of difficulty of the question, the level of difficulty should only be one of the following: easy, medium, hard.
#     - You should give the mark of the question. If the question is a multiple choice question, the mark is 1.
#     - You should give the type of the question, one of the following: multiple choice and short answer.
#     - You should give the answer of the question, you should give the answer in latex.
#         - if the question is a multiple choice question, the answer should be one of the choices (A, B, C, D). 
#         - if the question is a short answer question, the answer should be the answer to the question (just give the answer).
#         - You should try to give the answer for all subquestions.
#         - if there is no an explicit answer, you should give an empty string.
#     - Each image will only contain one question, but the question might have multiple sub-questions, you only need to give the topic, level of difficulty, mark and type of the main question once.
# Do not include any unnecessary information in the output which might includes titles, subtitles, etc.
# """

In [24]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    "openai:gpt-4o",
    timeout=None,
    max_tokens=512,
)

In [25]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  

agent = create_agent(
    model=llm,
    tools=[],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver()
)

In [26]:
question_file_id = "file-F4sS11gdBtzsCJKY4UHknK"
solution_file_id = "file-4HKYQitwgmU74U1Q1puaVN"
first5_solution_file_id = "file-4VtAqpn7qJfL6QZHKTs4jp"

# from openai import OpenAI
# client = OpenAI()

# upload = client.files.create(
#     file=open(".example/first5_solution.pdf", "rb"),
#     purpose="assistants"
# )
# print("Question file id:", upload.id)

# upload2 = client.files.create(
#     file=open(".example/solution.pdf", "rb"),
#     purpose="assistants"
# )
# print("Solution file id:", upload2.id)

In [27]:
from langchain.messages import HumanMessage

question = HumanMessage(content=[
        {"type": "text", "text":
            "Please ONLY summarize the content of the first page of the solution PDF. Do NOT scan the whole PDF. "
        },
        {
            "type": "file",
            "file_id": f"{question_file_id}",
            "mime_type": "application/pdf",
            "extras": {
                "filename": "paper.pdf",
                "description": "Original exam file containing all questions."
            }
        },
        {
            "type": "file",
            "file_id": f"{solution_file_id}",
            "mime_type": "application/pdf",
            "extras": {
                "filename": "solution.pdf",
                "description": "Solution preview."
            }
        },
    ])


In [28]:
for step in agent.stream(
    {"messages": [question]},
    {"configurable": {"thread_id": "1"}},  
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

# resp = agent.invoke({"messages": [question]})
# print(resp["messages"][-1].content)

================================ Human Message =================================

[{'type': 'text', 'text': 'Please ONLY summarize the content of the first page of the solution PDF. Do NOT scan the whole PDF. '}, {'type': 'file', 'file_id': 'file-F4sS11gdBtzsCJKY4UHknK', 'mime_type': 'application/pdf', 'extras': {'filename': 'paper.pdf', 'description': 'Original exam file containing all questions.'}}, {'type': 'file', 'file_id': 'file-4HKYQitwgmU74U1Q1puaVN', 'mime_type': 'application/pdf', 'extras': {'filename': 'solution.pdf', 'description': 'Solution preview.'}}]


APIConnectionError: Connection error.